In [1]:
import numpy as np
import re
def load_tests(file):
    originals = []
    with open(file) as f:
        for line in f:
            if line[0] == 'P':
                tmp = [i for i in re.split('POTENCJALNY_RYM:|\[\*\]', line) if i != '']    
            else:
                tmp = [i for i in re.split('RYM:|\[\*\]', line) if i != '']
            tmp = [j.split() for j in tmp]
            originals.append(tmp) 
    return originals

In [2]:
def load_supertags(file):
    d_tags = {}
    d_words = {}
    with open(file, 'r') as f:
        for line in f:
            t = line.split()[1]
            w = line.split()[0]
            d_words[w] = t
            if t not in d_tags:
                d_tags[t] = {}
            d_tags[t][w] = 1
    return d_tags, d_words

In [3]:
def load_superbazes(file):
    base = {}
    with open(file) as f:
        for line in f:
            word, base_word = line.split()
            base[word] = base_word
    return base

In [4]:
from tqdm.notebook import tqdm
def load_vectors(file):
    vectors = {}
    with open(file) as f:
        num_lines = sum(1 for line in open(file,'r'))
        for line in tqdm(f, total=num_lines):
            vec = line.split()
            x = np.array([float(i) for i in vec[1:]])
            vectors[vec[0]] = x
    return vectors

In [5]:
d_tags, d_words = load_supertags("supertags.txt")
originals = load_tests('rytmiczne_zdania_z_korpusu.txt')
base = load_superbazes('superbazy.txt')

In [6]:
vectors = load_vectors('poleval_base_vectors.txt')

In [7]:
sylabs = ['a','ą','e','ę','i','o','ó','u','y']
d_sylabs = ['i' + i for i in sylabs if i != 'i']
sylabs += [i.upper() for i in sylabs]

def count_sylabs(s):
    ans = []
    for w in s:
        c = 0
        for l in range( len(w) - 1):
            if w[l] in sylabs:
                c += 1
            if w[l] + w[l + 1] in d_sylabs:
                c -= 1
        if w[-1] in sylabs:
            c += 1
        if c != 0:
            ans.append(c)
    return ans

def get_rhym(w):
    c = 0
    #print(w)
    if w[-1] in sylabs:
        c += 1
    for l in range(len(w) - 2, 0, -1):
        if w[l] in sylabs:
            c += 1
        if w[l] + w[l + 1] in d_sylabs:
            c -= 1
        if c == 2:
            return w[l:]
    return w

def get_similar(words, og_word):
    best = ''
    m = 0
    tab = []
    o = og_word
    if og_word in base:
        og_word = base[og_word]
    else:
        return False
    for w in words:
        t = w
        if w in base:
            w = base[w]
        else:
            w = False
        if w in vectors and og_word in vectors:
            tmp = np.dot(vectors[w], vectors[og_word])
            #print(tmp, w)
            if tmp > m and o != t:
                best = t
                m = tmp
    if best != '':
        #print(o, best)
        #print(words)
        return best
    return False

In [8]:
def check(i, w):
    if sum(count_sylabs([w])) == sum(count_sylabs([i])) and get_rhym(w) == get_rhym(i) and i != w:
        return True
    return False

def find_best_rhym(w, tag, og_word):
    possible = [i for i in list(d_tags[tag].keys()) if check(i, w)] 
    #if len(possible) < 10:
        #print(possible)
    return get_similar(possible, og_word)

In [9]:
import copy
def gen_poem(every=False):
    sen = originals[np.random.choice(np.arange(len(originals)))]
    ans = copy.deepcopy(sen)
    id1_rhym = 0
    id2_rhym = 0
    for w in range(len(sen[0])):
        if sen[0][w].isalpha():
            id1_rhym = w
    for w in range(len(sen[1])):
        if sen[1][w].isalpha():
            id2_rhym = w
    
    first = sen[0][id1_rhym]
    if sen[1][id2_rhym] not in d_words:
        #print("Couldn't find tag, trying again", sen[1][id2_rhym])
        return gen_poem(every)
    second = find_best_rhym(first, d_words[sen[1][id2_rhym]], sen[1][id2_rhym])
    if not second:
        for i in range(10):
            if second:
                break
            tmp = get_similar(list(d_tags[d_words[first]].keys()), first)
            possible = list(d_tags[d_words[first]].keys())
            first = possible[np.random.choice(np.arange(len(possible)))]
            if tmp:
                first = tmp
            second = find_best_rhym(first, d_words[sen[1][id2_rhym]], sen[1][id2_rhym])
        if not second:
            #print("Couldn't find rhym, trying again", sen[0][id1_rhym])
            return gen_poem(every)
    
    ans[0][id1_rhym] = first
    ans[1][id2_rhym] = second
    
    for v in range(2):
        if v == 0:
            e = id1_rhym
        else:
            e = id2_rhym
        for w in range(e):
            if every and sen[v][w].isalpha():
                w2 = get_similar(list(d_tags[d_words[sen[v][w]]].keys()), sen[v][w])
                if w2:
                    ans[v][w] = w2
            
    return sen, ans

In [10]:
def cout(s1, s2):
    for i in range(2):
        print(' '.join(s1[i]))
    for i in range(2):
        print(' '.join(s2[i]))

In [11]:
for i in range(5):
    s1,s2 = gen_poem()
    cout(s1,s2)
    print()

czy gościom i kibicom wystarczy zachodnia
obwodnica poznania i autostrada ?
czy gościom i kibicom wystarczy zachodnia
obwodnica poznania i zarodnia ?

skoro chcemy nałożyć nowe obowiązki
, musimy wiedzieć , jakie za tym idą koszty .
skoro chcemy nałożyć nowe obowiązki
, musimy wiedzieć , jakie za tym idą nitrozwiązki .

jedyne co możemy , to zainspirować
lokalnych polityków i na nich naciskać .
jedyne co możemy , to zainspirować
lokalnych polityków i na nich interweniować .

każda firma dodaje do wyposażenia
kuchni co najmniej jedną blachę do pieczenia .
każda firma dodaje do wyposażenia
kuchni co najmniej jedną blachę do zaczerwienienia .

potem zarzucił sobie łopatę na ramię
i odszedł , nie spojrzawszy nawet na dziewczynkę .
potem zarzucił sobie łopatę na ramię
i odszedł , nie spojrzawszy nawet na amię .



In [12]:
for i in range(5):
    s1,s2 = gen_poem(True)
    cout(s1,s2)
    print()

powyższe stanowisko wynika z przyjętych
przez polskę zobowiązań międzynarodowych .
niniejsze obsadzanie rozmija ze przyjętych
poprzez litwę zadłużeń padniętych .

utwór z elementami grozy i suspensu
, jest także do pewnego stopnia formą żartu .
teledysk ze czynnikami fantastyki i suspensu
, przyrasta także ku interpretacyjnego orderu telepracą bezsensu .

prawie pewne , że będzie to wiceminister
obrony narodowej beata oczkowicz .
dobrze interpretacyjne , że będzie tamto wiceminister
piechoty gromadzkiej marzena gardemeister .

tam , gdzie kiedyś potrzebne było zezwolenie
na budowę , obecnie wystarczy zgłoszenie .
knajpek , skąd skądś konieczne zostało zezwolenie
pod obwodnicę , aktualnie odczeka doręczenie .

nie tylko warto , ale trzeba nad tą kwestią
podyskutować wspólnie , także z opozycją .
nie tylko warto , ależ widać pod tą kwestią
popracować wespół , także ze gestią .

